https://arxiv.org/pdf/2003.04030.pdf

### Архитектура

In [ ]:
import torch
from torch import nn

In [ ]:
class conv_bn_relu(nn.Module):

    def __init__(self, in_planes, out_planes, kernel_size, stride, padding, 
            has_bn=True, has_relu=True, groups=1):
        super(conv_bn_relu, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size,
                stride=stride, padding=padding, groups=groups)
        self.has_bn = has_bn
        self.has_relu = has_relu
        self.bn = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        def _func_factory(conv, bn, relu, has_bn, has_relu):
            def func(x):
                x = conv(x)
                if has_bn:
                    x = bn(x)
                if has_relu:
                    x = relu(x)
                return x
            return func 

        func = _func_factory(
                self.conv, self.bn, self.relu, self.has_bn, self.has_relu)

        return func(x)

In [ ]:
class PRM(nn.Module):

    def __init__(self, output_chl_num):
        super(PRM, self).__init__()

    def forward(self, x):

        return out

In [ ]:
test_input = torch.rand((10, 256, 32, 32))
prm = PRM(256)

### Данные: ключевые точки и heatmaps

https://github.com/caiyuanhao1998/RSN/blob/319961faa057ce49caffb274ae761a93e36f47d7/dataset/JointsDataset.py#L143

https://github.com/caiyuanhao1998/RSN/blob/319961faa057ce49caffb274ae761a93e36f47d7/dataset/JointsDataset.py#L216

In [ ]:
gaussian_kernels = [(11, 11), (9, 9), (7, 7), (5, 5)]
keypoint_num = 3
output_shape = (256, 256)
input_shape = (256, 256)

In [ ]:
import torch
import numpy as np
import cv2

def generate_heatmap(joints, valid, kernel=(7, 7)):
    heatmaps = np.zeros(
            (keypoint_num, *output_shape), dtype='float32')

    for i in range(keypoint_num):
        if valid[i] < 1:
            continue
        target_y = joints[i, 1] * output_shape[0] \
                / input_shape[0]
        target_x = joints[i, 0] * output_shape[1] \
                / input_shape[1]
        heatmaps[i, int(target_y), int(target_x)] = 1
        heatmaps[i] = cv2.GaussianBlur(heatmaps[i], kernel, 0)
        maxi = np.amax(heatmaps[i])
        if maxi <= 1e-8:
            continue
        heatmaps[i] /= maxi / 255

    return heatmaps 

In [ ]:
joints = np.array([[30, 30], [120, 120], [200, 200]]) #change to keypoint_num
valid = np.ones((keypoint_num, ))

labels_num = len(gaussian_kernels)
labels = np.zeros(
        (labels_num, keypoint_num, *output_shape))
for i in range(labels_num):
    labels[i] = generate_heatmap(
            joints, valid, kernel=gaussian_kernels[i])
labels = torch.from_numpy(labels).float()
labels.shape

torch.Size([4, 3, 256, 256])

In [ ]:
labels = labels.sum(dim=1)
labels = labels[0].numpy().astype(np.uint8)

In [ ]:
color = cv2.applyColorMap(labels, cv2.COLORMAP_JET)

In [ ]:
cv2.imwrite("output.png", color)

True

**keypoints_from_heatmaps**
https://mmpose.readthedocs.io/en/latest/_modules/mmpose/core/evaluation/top_down_eval.html#keypoints_from_heatmaps

heatmaps (np.ndarray[N, K, H, W]) – model predicted heatmaps.

center (np.ndarray[N, 2]) – Center of the bounding box (x, y).

scale (np.ndarray[N, 2]) – Scale of the bounding box wrt height/width


**Домашнее задание:**

Попробовать с использованием библиотеки FiftyOne выгрузить частичную выборку датасета ключевых точек людей из COCO и обучиться на этой подвыборке.

In [ ]:
https://docs.voxel51.com/integrations/coco.html
https://github.com/caiyuanhao1998/RSN
Pycoco tools